In [1]:
#1.开始前的工作-导入模块
import pandas as pd
import pymysql
from sklearn.cluster import KMeans  #聚类算法

In [2]:
#2.从MySQL里提取数据
#链接数据库
conn=pymysql.connect(host='localhost',user='root',passwd='111111',db='pysql',charset='utf8')
#创建游标
cursor=conn.cursor()
#sql语句
sql='select * from consumption_data'
#执行语句
cursor.execute(sql)
data=cursor.fetchall()  #获取所有数据
data

(('1', '27', '6', '232.61'),
 ('2', '3', '5', '1507.11'),
 ('3', '4', '16', '817.62'),
 ('4', '3', '11', '232.81'),
 ('5', '14', '7', '1913.05'),
 ('6', '19', '6', '220.07'),
 ('7', '5', '2', '615.83'),
 ('8', '26', '2', '1059.66'),
 ('9', '21', '9', '304.82'),
 ('10', '2', '21', '1227.96'),
 ('11', '15', '2', '521.02'),
 ('12', '26', '3', '438.22'),
 ('13', '17', '11', '1744.55'),
 ('14', '30', '16', '1957.44'),
 ('15', '5', '7', '1713.79'),
 ('16', '4', '21', '1768.11'),
 ('17', '93', '2', '1016.34'),
 ('18', '16', '3', '950.36'),
 ('19', '4', '1', '754.93'),
 ('20', '27', '1', '294.23'),
 ('21', '5', '1', '195.3'),
 ('22', '17', '3', '1845.34'),
 ('23', '12', '13', '1434.29'),
 ('24', '21', '3', '275.85'),
 ('25', '18', '5', '449.76'),
 ('26', '30', '21', '1628.68'),
 ('27', '4', '2', '1795.41'),
 ('28', '7', '12', '1786.24'),
 ('29', '18', '1', '679.44'),
 ('30', '60', '7', '5318.81'),
 ('31', '4', '22', '873.68'),
 ('32', '16', '1', '654.69'),
 ('33', '3', '2', '230.37'),
 ('34', 

In [3]:
#3.创建dataframe数据表
columnlist=['user_id','R','F','M']
rfm=pd.DataFrame(list(data),columns=columnlist)
# rfm.head() #查看前五行数据
# rfm.isnull() #查看是否有空值

In [4]:
#4.使用K-Means进行rfm模型聚类
#单独提取数据表中的F,R,M进行聚类
rfm_new=rfm[['R','F','M']]
rfm_new

R   F        M
0    27   6   232.61
1     3   5  1507.11
2     4  16   817.62
3     3  11   232.81
4    14   7  1913.05
5    19   6   220.07
6     5   2   615.83
7    26   2  1059.66
8    21   9   304.82
9     2  21  1227.96
10   15   2   521.02
11   26   3   438.22
12   17  11  1744.55
13   30  16  1957.44
14    5   7  1713.79
15    4  21  1768.11
16   93   2  1016.34
17   16   3   950.36
18    4   1   754.93
19   27   1   294.23
20    5   1    195.3
21   17   3  1845.34
22   12  13  1434.29
23   21   3   275.85
24   18   5   449.76
25   30  21  1628.68
26    4   2  1795.41
27    7  12  1786.24
28   18   1   679.44
29   60   7  5318.81
..   ..  ..      ...
910   9  14  1627.19
911  11  28   1154.7
912  17   1    33.58
913  15   9  1959.64
914  10  22  1581.14
915  27   5  1879.82
916  25   3   1142.4
917  15   1   174.64
918   5   2   638.81
919  19   4  1067.78
920  14  22  1345.92
921   6   3  1311.06
922  26   8   962.62
923  23   1   285.97
924  17  17  1651.68
925  16   3  1503.87
926   6  13  1506.48
927  22  24   625.12
928  28  17  1742.95
929  17  26  1292.21
930  16   7  1801.38
931   1  27   1585.1
932  17  27  7795.03
933   2  17  2341.93
934   1   5  1865.78
935  19   4  1163.08
936   9   7  1007.06
937  27   7  1322.94
938  30   4   860.41
939  22   1    776.7

[940 rows x 3 columns]

In [11]:
#设置随机数种子
seed=9
#设置KMeans模型参数，并带入指标数据进行拟合
clf=KMeans(n_clusters=8,random_state=seed)  #n_clusters=8划分为8类         -----模型初始化
#拟合
clf=clf.fit(rfm_new)  #fit()拟合数据
#查看KMeans聚类后结果（8个质心点的值）
D=pd.DataFrame(list(clf.cluster_centers_)) #查看聚类后结果
D

0          1            2
0  35.250000   9.750000  5759.110000
1  16.477987  10.597484   873.846352
2  16.269939  10.883436  1543.123620
3  17.068750   4.156250   204.536250
4  18.873418  11.613924  1226.188481
5  14.732484   9.751592   548.177325
6  16.605839  10.656934  1875.933285
7  14.500000  25.000000  7472.495000

In [7]:
#在原始表中对每个用户进行聚类后结果标记，目的是为了知道哪个用户ID在RFM模型中所属类别
rfm['label']=clf.labels_
rfm

user_id   R   F        M  label
0         1  27   6   232.61      3
1         2   3   5  1507.11      2
2         3   4  16   817.62      1
3         4   3  11   232.81      3
4         5  14   7  1913.05      6
5         6  19   6   220.07      3
6         7   5   2   615.83      5
7         8  26   2  1059.66      4
8         9  21   9   304.82      3
9        10   2  21  1227.96      4
10       11  15   2   521.02      5
11       12  26   3   438.22      5
12       13  17  11  1744.55      6
13       14  30  16  1957.44      6
14       15   5   7  1713.79      6
15       16   4  21  1768.11      6
16       17  93   2  1016.34      1
17       18  16   3   950.36      1
18       19   4   1   754.93      1
19       20  27   1   294.23      3
20       21   5   1    195.3      3
21       22  17   3  1845.34      6
22       23  12  13  1434.29      2
23       24  21   3   275.85      3
24       25  18   5   449.76      5
25       26  30  21  1628.68      2
26       27   4   2  1795.41      6
27       28   7  12  1786.24      6
28       29  18   1   679.44      5
29       30  60   7  5318.81      0
..      ...  ..  ..      ...    ...
910     913   9  14  1627.19      2
911     914  11  28   1154.7      4
912     915  17   1    33.58      3
913     916  15   9  1959.64      6
914     917  10  22  1581.14      2
915     918  27   5  1879.82      6
916     919  25   3   1142.4      4
917     920  15   1   174.64      3
918     921   5   2   638.81      5
919     922  19   4  1067.78      4
920     923  14  22  1345.92      4
921     924   6   3  1311.06      4
922     925  26   8   962.62      1
923     926  23   1   285.97      3
924     927  17  17  1651.68      2
925     928  16   3  1503.87      2
926     929   6  13  1506.48      2
927     930  22  24   625.12      5
928     931  28  17  1742.95      6
929     932  17  26  1292.21      4
930     933  16   7  1801.38      6
931     934   1  27   1585.1      2
932     935  17  27  7795.03      7
933     936   2  17  2341.93      6
934     937   1   5  1865.78      6
935     938  19   4  1163.08      4
936     939   9   7  1007.06      1
937     940  27   7  1322.94      4
938     941  30   4   860.41      1
939     942  22   1    776.7      1

[940 rows x 5 columns]

In [8]:
#查看RFM模型8个类别中的用户数量
c=rfm['label'].value_counts()  #value_counts()聚合统计
c

2    163
3    160
1    160
5    157
4    157
6    137
0      4
7      2
Name: label, dtype: int64

In [12]:
#第二种方式   查看RFM模型8个类别中的用户数量
r1=pd.Series(clf.labels_).value_counts()
r1

2    163
3    160
1    160
5    157
4    157
6    137
0      4
7      2
dtype: int64

In [13]:
r2=pd.DataFrame(clf.cluster_centers_)  #用dataframe格式查看所有质心点的值
r2

0          1            2
0  35.250000   9.750000  5759.110000
1  16.477987  10.597484   873.846352
2  16.269939  10.883436  1543.123620
3  17.068750   4.156250   204.536250
4  18.873418  11.613924  1226.188481
5  14.732484   9.751592   548.177325
6  16.605839  10.656934  1875.933285
7  14.500000  25.000000  7472.495000

In [14]:
r=pd.concat([r2,r1],axis=1)   #将质心点数据与每个类别统计数据合并成一个表
r

0          1            2    0
0  35.250000   9.750000  5759.110000    4
1  16.477987  10.597484   873.846352  160
2  16.269939  10.883436  1543.123620  163
3  17.068750   4.156250   204.536250  160
4  18.873418  11.613924  1226.188481  157
5  14.732484   9.751592   548.177325  157
6  16.605839  10.656934  1875.933285  137
7  14.500000  25.000000  7472.495000    2

In [15]:
r.columns=list(rfm_new.columns)+['类别数量']  #重新命名表头
r

R          F            M  类别数量
0  35.250000   9.750000  5759.110000     4
1  16.477987  10.597484   873.846352   160
2  16.269939  10.883436  1543.123620   163
3  17.068750   4.156250   204.536250   160
4  18.873418  11.613924  1226.188481   157
5  14.732484   9.751592   548.177325   157
6  16.605839  10.656934  1875.933285   137
7  14.500000  25.000000  7472.495000     2

In [ ]:
#以质心点的RFM中位数为基准划分类别
rmd=r['R'].median()
fmd=r['F'].median()
mmd=r['M'].median()

In [ ]:
r['lv']='label'
def lx(r):
    for i in range(8):
        if r.ix[i,'R']>=rmd and r.ix[i,'F']>=fmd and r.ix[i,'M']>=mmd:
            r.ix[i,'lv']='高价值客户'
        elif r.ix[i,'R']<rmd and r.ix[i,'F']>=fmd and r.ix[i,'M']>=mmd:
            r.ix[i,'lv']='重点保持客户'
        elif r.ix[i,'R']>=rmd and r.ix[i,'F']<fmd and r.ix[i,'M']>=mmd:
            r.ix[i,'lv']='重点发展客户'
        elif r.ix[i,'R']<rmd and r.ix[i,'F']<fmd and r.ix[i,'M']>=mmd:
            r.ix[i,'lv']='重点挽留客户'
        elif r.ix[i,'R']>=rmd and r.ix[i,'F']>=fmd and r.ix[i,'M']<mmd:
            r.ix[i,'lv']='一般价值客户'
        elif r.ix[i,'R']<rmd and r.ix[i,'F']>=fmd and r.ix[i,'M']<mmd:
            r.ix[i,'lv']='一般保持客户'
        elif r.ix[i,'R']>=rmd and r.ix[i,'F']<fmd and r.ix[i,'M']<mmd:
            r.ix[i,'lv']='一般发展客户'
        elif r.ix[i,'R']<rmd and r.ix[i,'F']<fmd and r.ix[i,'M']<mmd:
            r.ix[i,'lv']='潜在客户'
    return r
print(lx(r))

In [ ]:
#详细输出原始数据集其类别
list1=[]
for n in rfm['label']:
    if n==0:
        list1.append('重点发展客户')
    elif n==1:
        list1.append('潜在客户')
    elif n==2:
        list1.append('重点保持客户')
    elif n==3:
        list1.append('一般发展客户')
    elif n==4:
        list1.append('一般价值客户')
    elif n==5:
        list1.append('潜在客户')
    elif n==6:
        list1.append('高价值客户')
    elif n==7:
        list1.append('重点保持客户')


In [ ]:
rfm['聚类类别']=list1
rfm

In [ ]:
rfm.to_excel('RFM模型结果.xls')